### instantiating services

In [1]:
from datetime import datetime
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
from abis import economyAbi
import re
from entities.project import Project
from entities.economy import Economy
from abis import nativeProjectAbi
import concurrent.futures

cred = credentials.Certificate('trustless.json')
initialize_app(cred)
db = firestore.client()
networks=db.collection("networks")
ceva=networks.document("Etherlink-Testnet").get()
economy_address = ceva.to_dict()['economyAddress']
print("economy address :" + str(economy_address))
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
projects = []
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

economy_abi=re.sub(r'\n+', ' ', economyAbi).strip()
economy_contract = web3.eth.contract(address=economy_address, abi=economy_abi)
projects_collection=db.collection('iProjectsEtherlink-Testnet')
economy=Economy(economy_contract)

native_project_abi=re.sub(r'\n+', ' ', nativeProjectAbi).strip()
project_contract = web3.eth.contract(address=economy_address, abi=native_project_abi)

economy address :0xBBdfB8ad5ea5DE3eD6D2CF3A78dc13E89509af54
node connected


In [ ]:
import time
current_block=web3.eth.block_number
counter=0
start_block=current_block-10
processed_transactions = set()
heartbeat=0

while True:
    heartbeat+=1
    logs = web3.eth.get_logs({
        "fromBlock": start_block,
        "toBlock": web3.eth.block_number,
        "address": economy_address,
        "topics": [
            web3.keccak(text="NewProject(address,string,address,address,string,string,string)").hex(),  # Removed spaces
            None  # Added None to match any contractAddress
        ]
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            continue  # Skip duplicate
        print("new project detected !")
        decoded_event = economy_contract.events.NewProject().process_log(log)
        processed_transactions.add(tx_hash)
        address = decoded_event['args']['contractAddress']
        projectName = decoded_event['args']['projectName']
        contractor = decoded_event['args']['contractor']
        arbiter = decoded_event['args']['arbiter']
        termsHash = decoded_event['args']['termsHash']
        repo = decoded_event['args']['repo']
        transactionHash = decoded_event['transactionHash']
        description = decoded_event['args']['description']
        p: Project = Project(address, projectName, contractor,
                                arbiter, termsHash, repo, description)
        projects_collection.document(p.address).set(p.serialize())
        processed_transactions.add(tx_hash)
    if heartbeat%10==0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)
    

In [ ]:
import importlib
import entities.project
from entities.project import Project, ProjectEvents
importlib.reload(entities.project)

In [20]:
project_addresses = [doc.id for doc in projects_collection.stream()]
project_addresses

['0x379080EE8941CE5Ae5560d5E32A4CEBBaC7C6b92']

In [ ]:
def listen_to_project_events(contract, contract_address):
    global processed_transactions
    
    # Define the topics for all events
    event_signatures = {
        "SetParties": web3.keccak(text="SetParties(address,address,string)").hex(),
        "SendFunds": web3.keccak(text="SendFunds(address,uint256)").hex(),
        "ContractorPaid": web3.keccak(text="ContractorPaid(address,uint256)").hex(),
        "ContributorWithdrawn": web3.keccak(text="ContributorWithdrawn(address,uint256)").hex(),
        "ProjectDisputed": web3.keccak(text="ProjectDisputed(address)").hex(),
        "ProjectClosed": web3.keccak(text="ProjectClosed(address)").hex(),
        "ContractSigned": web3.keccak(text="ContractSigned(address)").hex(),
        "ArbitrationDecision": web3.keccak(text="ArbitrationDecision(address,uint256,string)").hex(),
    }

    # Get logs for the current contract
    logs = web3.eth.get_logs({
        "fromBlock": web3.eth.block_number-3,
        "toBlock": web3.eth.block_number,
        "address": contract_address,
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip already processed transactions

        try:
            event_type = None
            for event_name, topic_hash in event_signatures.items():
                print("got one!")
                if log["topics"][0].hex() == topic_hash:
                    event_type = event_name
                    break

            if not event_type:
                print("Not an avent type")
                continue  # Skip logs that don't match any defined events

            # Decode and process the event
            decoded_event = getattr(contract.events, event_type)().process_log(log)
            event_data = getattr(ProjectEvents, event_type)(decoded_event)

            print(f"Event detected in contract {contract_address} ({event_type}):", event_data)

        except Exception as e:
            print(f"Error processing log for contract {contract_address}: {e}")

        # Mark the transaction as processed
        processed_transactions.add(tx_hash)


In [3]:
from eth_keys import keys
from eth_utils import to_checksum_address
import os

def generate_eth_address():
    # Generate a random private key
    private_key = keys.PrivateKey(os.urandom(32))
    public_key = private_key.public_key

    # Generate the Ethereum address
    eth_address = to_checksum_address(public_key.to_address())

    return private_key, eth_address

# Generate and print 5 Ethereum addresses with their private keys
for i in range(5):
    private_key, eth_address = generate_eth_address()
    print(f"Address {i + 1}: {eth_address}")
    print(f"Private Key {i + 1}: {private_key}\n")

Address 1: 0xc5E9Cf03744BB2D29cd23B29B1dBe5411dff2a8c
Private Key 1: 0xa0e286b238abfbabce90331211c4f5ea65d59e91bfb2aa8b605eccdf9eef7500

Address 2: 0xe6429070e21325E34963a5147C4ff93441554859
Private Key 2: 0xab406f179b756406809c77b628bc0ba7aefbb74edc07083644bd3d726677c0e2

Address 3: 0xbA3b4E0D9B83348553328A8F0098B3a0190E4432
Private Key 3: 0x1b5481a7b44157d6a63cbc3a5206ca4b208cb748a4f42e509adfabe09ded2292

Address 4: 0xf62f56f07f511417245a7eD9BCD7e6429954e807
Private Key 4: 0x9293398dc74c25ca969fe43d4678cc919247f4501817a2f472c8fbb76aa8e7a7

Address 5: 0x305Ad7F8Ef0555ED2f00D870179A5fAfC67b7c8b
Private Key 5: 0xcdfdc80dd79e9059a8e0f3ccb8d1d6a94f0f8c50328389fa82d9a8f5f9665121



In [ ]:
while True:
    heartbeat += 1

    for address in project_addresses:
        # Get contract instance
        contract = web3.eth.contract(address=address, abi=native_project_abi)
        try:
            listen_to_project_events(contract, address)
        except Exception as e:
            
            print("Errrorrr "+e)

    if heartbeat % 10 == 0:
        print(f"Heartbeat: {heartbeat}")

    time.sleep(3)

In [28]:
import entities.services

In [ ]:
import requests
dorg_homebase_channel_id = "804689936388325376"

def send_discord_message(msg, channel_id):
    global discord_bot_token
    url = "https://discordapp.com/api/channels/"+channel_id+"/messages"
    headers = {
        "Authorization": "Bot " + discord_bot_token
    }
    body = {
        "content": msg
    }
    print("before sending "+msg)
    response = requests.request("POST", url, headers=headers, data=body)
    return {"data": str("Status code "+str(response.status_code))}

In [31]:
send_discord_message(channel_id=dorg_homebase_channel_id, msg="INDEXER IS DOWN ON TRUSTLESS!")

before sending INDEXER IS DOWN ON TRUSTLESS!


{'data': 'Status code 200'}

In [33]:
from dotenv import dotenv_values
env_variables = dotenv_values(".env")

In [ ]:
my_variable = env_variables["discord_bot_token"]
print(my_variable)